In [1]:
from IPython.display import Image 
import _init_paths

In [3]:
from ocean_lib.ocean.ocean import Ocean
from ocean_lib.config import Config

config = Config('config.ini')
ocean = Ocean(config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

config.network_url = 'https://rinkeby.infura.io/v3/d163c48816434b0bbb3ac3925d6c6c80'
config.block_confirmations = 0
config.metadata_cache_uri = 'https://aquarius.oceanprotocol.com'
config.provider_url = 'https://provider.rinkeby.oceanprotocol.com'


In [4]:
import os
from ocean_lib.web3_internal.wallet import Wallet

wallet = Wallet(ocean.web3, os.getenv('MY_TEST_KEY'), transaction_timeout=20, block_confirmations=config.block_confirmations)

print(f"public address = '{wallet.address}'")

public address = '0x2338e4e94AEe1817701F65f2c751f7c844b0e43b'


In [5]:
from ocean_lib.web3_internal.currency import to_wei

# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('DEFI0', 'DEFI0', wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(wallet.address, to_wei(100), wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

ALG_datatoken.address = '0x325bA298E241d4B8d68A6F6E4985935e57Cde939'


In [7]:
from ocean_lib.web3_internal.currency import pretty_ether_and_wei
print(f"Pool has {pretty_ether_and_wei(ALG_datatoken.balanceOf(wallet.address), ALG_datatoken.symbol())}.")

Pool has 1E+2 DEFI0 (100000000000000000000 wei).


## Publish algorithm

In [9]:
# Important Parameters
image: str = "algovera/algo_dockers" 
tag: str = "defi-borrowing"
url: str = "https://raw.githubusercontent.com/AlgoveraAI/DeFi-borrowing-cost-prediction/main/algo-c2d.py"
name: str = "DeFi Borrowing Cost Prediction"
author: str = "AlgoveraAI"

In [10]:
# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.1",
            "container": {
              "entrypoint": "python $ALGO",
              "image": image,
              "tag": tag
            }
        },
        "files": [
    {
        "url": url,
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": name, "author": author, "license": "CC0",
    "dateCreated": "2022-01-31T15:00:00Z"
    }
}

ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

In [11]:
# Set up a service provider. We'll use this same provider for ALG
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
provider_url = DataServiceProvider.get_url(ocean.config)
# returns "http://localhost:8030"

# Calc DATA service compute descriptor
from ocean_lib.services.service import Service
from ocean_lib.common.agreements.service_types import ServiceTypes

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=ALG_service_attributes
)

# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=wallet,
  services=[ALG_access_service],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

signing message with nonce 0: 0x325bA298E241d4B8d68A6F6E4985935e57Cde939, account=0x2338e4e94AEe1817701F65f2c751f7c844b0e43b
ALG did = 'did:op:325bA298E241d4B8d68A6F6E4985935e57Cde939'


In [12]:
# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=wallet,
  service_descriptors=[ALG_access_service_descriptor],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

NameError: name 'ALG_access_service_descriptor' is not defined

At this point you will need to request that the data provider approves your code as a trusted algorithm on the dataset. For example, if you reach out to Algovera about one of our datasets, we will approve your request quickly.